---
# Problem 4

In [28]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 15 04:23:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    34W / 250W |  15963MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, LSTM, GRU, Input, BatchNormalization, Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import HeNormal, HeUniform
from tensorflow.keras.callbacks import CSVLogger
import random
import matplotlib.pyplot as plt
import seaborn as sns
from keras import backend

import pickle
!pip install librosa 
import librosa
import librosa.display

%matplotlib inline 

%tensorflow_version 2.x

In [30]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


### 4.1 Reading and preprocessing data

In [31]:
with open('drive/MyDrive/e533_hw4/models/hw4_trs.pkl', 'rb') as f:
  trs = pickle.load(f)
trs.shape

(500, 16180)

In [32]:
trs_stft = []
for s in trs:
  S = librosa.stft(s, n_fft=1024, hop_length=512)
  trs_stft.append(np.abs(S).T)
del S
trs_stft = np.array(trs_stft)
trs_stft.shape

(500, 32, 513)

In [33]:
original_shape = trs_stft[0].shape
# original_shape

In [34]:
pad = np.zeros(shape=(45-32,513))

In [37]:
# create Class:positive samples
samples_positive = []
for speaker in range(0, trs_stft.shape[0], 10):
  i = speaker
  for i in range(10):
    for j in range(i+1, 10):
      samples_positive.append([ np.vstack([trs_stft[speaker+i], pad]), np.vstack([trs_stft[speaker+j], pad])])
      
samples_positive = np.array(samples_positive)

labels_positive = np.ones(shape=(samples_positive.shape[0],)).astype('int32')

print(f'Shapes: X:{samples_positive.shape}; y:{labels_positive.shape}')

Shapes: X:(2250, 2, 45, 513); y:(2250,)


In [ ]:
# # create Class:positive samples
# samples_positive = []
# for speaker in range(0, trs_stft.shape[0], 10):
#   i = speaker
#   for i in range(10):
#     for j in range(i+1, 10):
#       temp_i = np.vstack((trs_stft[speaker+i], np.zeros(shape=(45-original_shape[0], original_shape[1])) ))
#       temp_j = np.vstack((trs_stft[speaker+j], np.zeros(shape=(45-original_shape[0], original_shape[1])) ))
#       samples_positive.append([temp_i, temp_j])
      
# samples_positive = np.array(samples_positive)

# labels_positive = np.ones(shape=(samples_positive.shape[0],)).astype('int32')

# print(f'Shapes: X:{samples_positive.shape}; y:{labels_positive.shape}')

In [39]:
samples_negative = []
for speaker in range(0, trs_stft.shape[0], 10):
  # speakers_index = range(speaker, speaker+10)
  other_speakers_index = list(range(0,i)) + list(range(i+10, trs_stft.shape[0]))
  # create 45 sample per speaker
  for i in range(10):
    for j in range(i+1, 10):
      randindex = random.randint(0,len(other_speakers_index)-1)
      samples_negative.append([np.vstack([trs_stft[speaker+i], pad]),np.vstack([trs_stft[other_speakers_index[randindex]], pad]) ])

samples_negative = np.array(samples_negative)

labels_negative = np.zeros(shape=(samples_negative.shape[0],)).astype('int32')

print(f'Shapes: X:{samples_negative.shape}; y={labels_negative.shape}')

Shapes: X:(2250, 2, 45, 513); y=(2250,)


In [ ]:
# samples_negative = []
# for speaker in range(0, trs_stft.shape[0], 10):
#   # speakers_index = range(speaker, speaker+10)
#   other_speakers_index = list(range(0,i)) + list(range(i+10, trs_stft.shape[0]))
#   # create 45 sample per speaker
#   for i in range(10):
#     for j in range(i+1, 10):
#       randindex = random.randint(0,len(other_speakers_index)-1)
      
#       temp_i = np.vstack((trs_stft[speaker+i] , np.zeros(shape=(45-original_shape[0], original_shape[1])) ))
#       temp_j = np.vstack((trs_stft[other_speakers_index[randindex]], np.zeros(shape=(45-original_shape[0], original_shape[1])) ))
#       samples_negative.append([temp_i, temp_j ])

# samples_negative = np.array(samples_negative)

# labels_negative = np.zeros(shape=(samples_negative.shape[0],)).astype('int32')

# print(f'Shapes: X:{samples_negative.shape}; y={labels_negative.shape}')

In [40]:
x_train = np.concatenate([samples_positive, samples_negative], axis=0)
y_train = np.concatenate([labels_positive, labels_negative], axis=0)
# y_train = to_categorical(y_train)
print(f'Shapes: x_train:{x_train.shape}; y_train:{y_train.shape}')

Shapes: x_train:(4500, 2, 45, 513); y_train:(4500,)


In [41]:
y_train = np.asarray(y_train).astype('int32').reshape((-1,1))
# y_test = np.asarray(test_labels).astype('float32').reshape((-1,1))

In [42]:
np.array_equal(x_train[2251,0],x_train[2251,1])

False

### 4.2 Create siamese model

In [43]:
def eudlidean_dist(maps):
  # map1, map2 = maps
  return backend.sqrt(backend.sum(backend.square(maps[0] - maps[1]), axis=-1))

# create RNN to run within the Siamese model
def create_rnn_model(num_rnn_layers = 3, units_per_rnn_layer=512, layer_type='LSTM', units_dense_out=513, rate_dropout=0.2):
  assert layer_type in ['LSTM', 'GRU']
  model = keras.Sequential()
  # model.add(Flatten())
  layer_name_generic = 'layer_'
  for i in range(num_rnn_layers):
    if layer_type == 'LSTM':
      model.add(LSTM(units_per_rnn_layer, return_sequences=True, kernel_initializer=tf.keras.initializers.HeUniform(), name=layer_name_generic+layer_type+'_'+str(i)))
    elif layer_type == 'GRU':
      model.add(GRU(units_per_rnn_layer, return_sequences=True, kernel_initializer=tf.keras.initializers.HeUniform(), name=layer_name_generic+layer_type+'_'+str(i)))
    model.add(Dropout(rate_dropout))
  model.add(Dense(units_dense_out, name='dense_out'))
  return model

# def create_siamese_model(input_shape, num_rnn_layers = 3, units_per_rnn_layer=512, layer_type='LSTM', units_dense_out=513, rate_dropout=0.2):



In [44]:
INPUT_SHAPE = x_train[0,0,:].shape

model_rnn = create_rnn_model(num_rnn_layers = 4, units_per_rnn_layer=1024, units_dense_out=513)

input_layers = [keras.Input(shape=INPUT_SHAPE), keras.Input(shape=INPUT_SHAPE)]
feature_map0, feature_map1 = model_rnn(input_layers[0]), model_rnn(input_layers[1])

lambda_layer = layers.Lambda(lambda maps: eudlidean_dist(maps=maps))
# lambda_layer = layers.Lambda(lambda maps: backend.abs(maps[0] - maps[1]))

dist_maps = lambda_layer([feature_map0, feature_map1])
print(dist_maps)
model_siamese = keras.Model(
    inputs=input_layers, 
    outputs=Dense(1, activation='sigmoid', kernel_initializer=HeUniform(), bias_initializer=HeUniform(), name='dense_predictor')(dist_maps)
    )

model_siamese.compile(
    loss = BinaryCrossentropy(),
    optimizer=Adam(learning_rate=1e-05),
    metrics=['accuracy']
    )

model_siamese.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 45), dtype=tf.float32, name=None), name='lambda_2/Sqrt:0', description="created by layer 'lambda_2'")
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 45, 513)]    0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 45, 513)]    0           []                               
                                                                                                  
 sequential_2 (Sequential)      (None, 45, 513)      32003585    ['input_5[0][0]',                
                                                                  'input_6[0][0]']                
                                           

In [45]:
model_rnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_LSTM_0 (LSTM)         (None, 45, 1024)          6299648   
                                                                 
 dropout_8 (Dropout)         (None, 45, 1024)          0         
                                                                 
 layer_LSTM_1 (LSTM)         (None, 45, 1024)          8392704   
                                                                 
 dropout_9 (Dropout)         (None, 45, 1024)          0         
                                                                 
 layer_LSTM_2 (LSTM)         (None, 45, 1024)          8392704   
                                                                 
 dropout_10 (Dropout)        (None, 45, 1024)          0         
                                                                 
 layer_LSTM_3 (LSTM)         (None, 45, 1024)         

### 4.3 Training the model

In [46]:
# tf.config.run_functions_eagerly(True)


In [47]:
model_siamese_csv_log = CSVLogger("model_siamese_csv_log_eu.csv")

model_siamese_fit = model_siamese.fit(x=[x_train[:,0, :], x_train[:, 1, :]],
                  y=y_train,
                  batch_size=512,
                  epochs=100,
                  verbose=2,
                  callbacks=[model_siamese_csv_log]
                  )

!cp model_siamese_csv_log_eu.csv "drive/MyDrive/e533_hw4/Part4/"

Epoch 1/100
9/9 - 21s - loss: 0.9399 - accuracy: 0.4889 - 21s/epoch - 2s/step
Epoch 2/100
9/9 - 11s - loss: 0.8402 - accuracy: 0.4962 - 11s/epoch - 1s/step
Epoch 3/100
9/9 - 11s - loss: 0.7726 - accuracy: 0.5053 - 11s/epoch - 1s/step
Epoch 4/100
9/9 - 11s - loss: 0.7364 - accuracy: 0.5351 - 11s/epoch - 1s/step
Epoch 5/100
9/9 - 11s - loss: 0.7103 - accuracy: 0.5547 - 11s/epoch - 1s/step
Epoch 6/100
9/9 - 11s - loss: 0.6920 - accuracy: 0.5804 - 11s/epoch - 1s/step
Epoch 7/100
9/9 - 11s - loss: 0.6782 - accuracy: 0.6051 - 11s/epoch - 1s/step
Epoch 8/100
9/9 - 11s - loss: 0.6574 - accuracy: 0.6384 - 11s/epoch - 1s/step
Epoch 9/100
9/9 - 11s - loss: 0.6451 - accuracy: 0.6496 - 11s/epoch - 1s/step
Epoch 10/100
9/9 - 11s - loss: 0.6217 - accuracy: 0.6778 - 11s/epoch - 1s/step
Epoch 11/100
9/9 - 11s - loss: 0.6034 - accuracy: 0.6840 - 11s/epoch - 1s/step
Epoch 12/100
9/9 - 11s - loss: 0.5825 - accuracy: 0.7044 - 11s/epoch - 1s/step
Epoch 13/100
9/9 - 11s - loss: 0.5624 - accuracy: 0.7271 - 11

Validation

In [48]:
with open('drive/MyDrive/e533_hw4/models/hw4_tes.pkl', 'rb') as f:
  tes = pickle.load(f)

tes.shape

(200, 22631)

In [52]:
tes_stft = []
for s in tes:
  S = librosa.stft(s, n_fft=1024, hop_length=512)
  tes_stft.append(np.abs(S).T)
del S
tes_stft = np.array(tes_stft)
tes_stft.shape

(200, 45, 513)

In [53]:
# create validation Class:positive samples
Valid_positive = []
for speaker in range(0, tes_stft.shape[0], 10):
  i = speaker
  for i in range(10):
    for j in range(i+1, 10):
      Valid_positive.append([ tes_stft[speaker+i], tes_stft[speaker+j] ])
      
Valid_positive = np.array(Valid_positive)

Valid_labels_positive = np.ones(shape=(Valid_positive.shape[0],)).astype('int32')

print(f'Shapes: X:{Valid_positive.shape}; y:{Valid_labels_positive.shape}')

Shapes: X:(900, 2, 45, 513); y:(900,)


In [54]:
Valid_negative = []
for speaker in range(0, tes_stft.shape[0], 10):
  # speakers_index = range(speaker, speaker+10)
  other_speakers_index = list(range(0,i)) + list(range(i+10, tes_stft.shape[0]))
  # create 45 sample per speaker
  for i in range(10):
    for j in range(i+1, 10):
      randindex = random.randint(0,len(other_speakers_index)-1)
      Valid_negative.append([tes_stft[speaker+i],tes_stft[other_speakers_index[randindex]] ])

Valid_negative = np.array(Valid_negative)

Valid_labels_negative = np.zeros(shape=(Valid_negative.shape[0],)).astype('int32')

print(f'Shapes: X:{Valid_negative.shape}; y={Valid_labels_negative.shape}')

Shapes: X:(900, 2, 45, 513); y=(900,)


In [55]:
x_validation = np.concatenate([Valid_positive, Valid_negative], axis=0)
y_validation = np.concatenate([Valid_labels_positive, Valid_labels_negative], axis=0)
y_validation = np.asarray(y_validation).astype('int32').reshape((-1,1))


In [57]:
model_siamese.evaluate(x=[x_validation[:,0,],x_validation[:,1,]], y=y_validation, batch_size=40)

45/45 [==============================] - 6s 80ms/step - loss: 1.4640 - accuracy: 0.6739


[1.4640339612960815, 0.6738888621330261]